<a href="https://colab.research.google.com/github/myazzeh/NLP-Course/blob/main/Word_2_Vec/NLP_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Install gensim Library**

In [31]:
!pip install gensim

#**Import Required Libraries:**
1- gensim.model.Word2Vec \
2- gensim.model.Doc2Vec \
3- gensim.model.FastText \

In [32]:
import gensim
from gensim.models import Word2Vec, Doc2Vec, FastText
from sklearn.model_selection import train_test_split
import pandas as pd

In [33]:
df = pd.read_csv('Spam.csv')
df.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [34]:
X= df['Message']
y= df['Category']
y.replace('spam', 1, inplace=True)
y.replace('ham', 0, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [35]:
x_train = [txt.split() for txt in x_train]
x_test = [txt.split() for txt in x_test]

In [36]:
x_train[0:3]

[['As', 'usual..iam', 'fine,', 'happy', '&amp;', 'doing', 'well..:)'],
 ['No',
  "you'll",
  'just',
  'get',
  'a',
  'headache',
  'trying',
  'to',
  'figure',
  'it',
  'out.',
  'U',
  'can',
  'trust',
  'me',
  'to',
  'do',
  'the',
  'math.',
  'I',
  'promise.',
  'O:-)'],
 ['Tired.', 'I', "haven't", 'slept', 'well', 'the', 'past', 'few', 'nights.']]

#**Build Word2Vec Model**

In [37]:
model= Word2Vec(sentences= x_train, vector_size=10, window=3, sg=1, epochs=50)

In [38]:
vocab = model.wv.index_to_key
print(vocab)
print(f"length of vocab is {len(vocab)}")

['to', 'you', 'I', 'a', 'the', 'and', 'in', 'i', 'is', 'u', 'for', 'my', 'of', 'me', 'your', 'have', 'on', '2', 'that', 'it', 'are', 'call', 'or', 'be', 'with', 'not', 'will', 'at', 'get', 'can', 'so', 'U', 'ur', "I'm", 'but', 'You', '4', 'from', 'do', '&lt;#&gt;', 'up', 'if', 'just', 'all', 'we', 'got', 'when', 'know', 'come', 'out', 'was', '.', 'go', 'like', 'this', 'now', '?', 'am', 'want', '...', 'send', 'by', 'Call', 'need', 'How', 'n', 'been', 'as', 'then', 'time', "I'll", 'only', 'about', 'what', 'our', 'But', 'still', 'one', 'If', 'he', 'text', 'going', 'good', 'its', 'No', 'no', 'has', 'Just', 'see', "don't", 'Your', 'how', 'Do', 'So', 'any', 'We', 'love', 'back', 'take', 'her', 'there', 'd', 'day', 'think', 'free', 'ü', 'an', 'some', '&', 'r', 'home', 'My', 'dont', 'Ok', 'she', 'tell', 'they', 'me.', 'mobile', 'A', 'What', 'later', '-', 'make', 'And', '&amp;', 'Have', 'phone', 'much', 'FREE', 'you.', 'him', 'now.', 'more', "i'm", 'who', 'ask', 'Are', 'meet', 'The', 'give', 'T

In [39]:
model.wv['message']

array([ 0.80300736, -0.8707511 , -0.5660014 , -0.34032258,  0.45615298,
        0.20378454,  1.4993888 ,  0.8710505 ,  0.21312495, -0.6899118 ],
      dtype=float32)

In [40]:
# How to create vector for each sentence
model.wv.similar_by_word('message', topn=10)

[('link', 0.9310705661773682),
 ('sent', 0.9203552007675171),
 ('mid', 0.9202043414115906),
 ('For', 0.9160977005958557),
 ('email', 0.9067047834396362),
 ('unsubscribe', 0.8996850848197937),
 ('mail', 0.894523561000824),
 ('details', 0.8930008411407471),
 ('sms', 0.88742995262146),
 ('ready', 0.8858189582824707)]

In [41]:
model.wv.most_similar('message', topn=10)

[('link', 0.9310705661773682),
 ('sent', 0.9203552007675171),
 ('mid', 0.9202043414115906),
 ('For', 0.9160977005958557),
 ('email', 0.9067047834396362),
 ('unsubscribe', 0.8996850848197937),
 ('mail', 0.894523561000824),
 ('details', 0.8930008411407471),
 ('sms', 0.88742995262146),
 ('ready', 0.8858189582824707)]

#**Create Sentence Vector**

In [42]:
import numpy as np
def GenerateSentencevector(w2v_model, data):
  sent_vect=[]
  for sentence in data:
    temp= np.zeros(10)
    for token in sentence:
      if token in model.wv:
        temp = temp + np.array(w2v_model.wv[token])
    sent_vect.append(temp)

  return np.array(sent_vect)

Xtrain = GenerateSentencevector(model, x_train)
Xtest  = GenerateSentencevector(model, x_test)

#**Build classification model**

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(Xtrain, y_train)
prd_lr = lr.predict(Xtest)
print(f'Accuracy of logisitic regression is {accuracy_score(y_test, prd_lr)}')

Accuracy of logisitic regression is 0.9683014354066986


In [49]:
nvb = GaussianNB()
nvb.fit(Xtrain, y_train)
prd_nvb = nvb.predict(Xtest)
print(f'Accuracy of naive Bays is {accuracy_score(y_test, prd_nvb)}')

Accuracy of naive Bays is 0.9246411483253588


In [48]:
# build classification model using Neural Networks

from keras.layers import Input, Dense
from keras.models import Sequential

nn = Sequential(
    [
        Input(shape = (10,)),
        Dense(units= 8, activation= 'relu'),
        Dense(units= 4, activation= 'relu'),
        Dense(units= 1, activation= 'sigmoid'),

    ]
)

nn.compile(optimizer ='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(Xtrain, y_train, epochs=20)
nn.evaluate(Xtest, y_test)

Epoch 1/20
122/122 [==============================] - 2s 4ms/step - loss: 0.5710 - accuracy: 0.8310
Epoch 2/20
122/122 [==============================] - 1s 4ms/step - loss: 0.4066 - accuracy: 0.8641
Epoch 3/20
122/122 [==============================] - 0s 3ms/step - loss: 0.3184 - accuracy: 0.8633
Epoch 4/20
122/122 [==============================] - 0s 3ms/step - loss: 0.2789 - accuracy: 0.8633
Epoch 5/20
122/122 [==============================] - 0s 3ms/step - loss: 0.2539 - accuracy: 0.8633
Epoch 6/20
122/122 [==============================] - 0s 3ms/step - loss: 0.2362 - accuracy: 0.8633
Epoch 7/20
122/122 [==============================] - 0s 4ms/step - loss: 0.2220 - accuracy: 0.8633
Epoch 8/20
122/122 [==============================] - 0s 3ms/step - loss: 0.2116 - accuracy: 0.8633
Epoch 9/20
122/122 [==============================] - 0s 3ms/step - loss: 0.2037 - accuracy: 0.8633
Epoch 10/20
122/122 [==============================] - 0s 3ms/step - loss: 0.1970 - accuracy: 0.8633

[0.13278846442699432, 0.9724880456924438]